See the google collaboratory for more details:
https://colab.research.google.com/drive/1mCbcWxvmARx1KTDbYHGVTNOa66Z4uCoJ?usp=sharing

In [2]:
# start meshcat server
server_args = []
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

In [6]:
import numpy as np
from functools import partial

from ipywidgets import FloatSlider, Layout, Dropdown, Textarea
from IPython.display import display, HTML, SVG

from pydrake.common.jupyter import process_ipywidget_events
from pydrake.systems.framework import BasicVector, VectorSystem

from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve

from pydrake.all import (
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator, InverseKinematics)

from pydrake.math import RigidTransform, RotationMatrix, RollPitchYaw
from pydrake.all import RigidTransform, RotationMatrix
from pydrake.geometry import Cylinder
from pydrake.multibody.tree import UnitInertia, SpatialInertia, JointIndex
from pydrake.multibody.plant import CoulombFriction
#from pydrake.multibody.inverse_kinematics import InverseKinematics
from PandaInverseKinematics import PandaInverseKinematics

In [4]:
def AddCylinder(plant, name, radius, length):

  mass = 1
  mu = 1
  color = [0.486, 0.305, 0.756,1]

  shape = Cylinder(radius, length)
  instance = plant.AddModelInstance(name)
  inertia = UnitInertia.SolidCylinder(shape.radius(), shape.length())

  body = plant.AddRigidBody(name, instance, SpatialInertia(mass=mass,p_PScm_E=np.array([0., 0., 0.]), G_SP_E=inertia))
    
  plant.RegisterCollisionGeometry(body, RigidTransform(), shape, name, CoulombFriction(mu, mu))

  plant.RegisterVisualGeometry(body, RigidTransform(), shape, name, color)

  return instance

def PoseWidget(publishing_system, root_context, callback):
  ''' create the slider widget that will callback with the desired position to the IK solver'''

  vars = ['x (m)','y (m)','z (m)','roll (rad)','pitch (rad)','yaw (rad)']
  desired_pose = np.array([0.3,0.5,0.3,0,np.pi/2,np.pi/2]) # initial slider (and hand) pose

  publishing_context = publishing_system.GetMyContextFromRoot(root_context) # Returns the mutable subsystem context for this system, given a root context.
  
  callback(root_context, desired_pose)
  publishing_system.Publish(publishing_context)

  def slider_callback(change, index):
    desired_pose[index] = change.new
    callback(root_context, desired_pose)
    publishing_system.Publish(publishing_context)

  slider_widgets = []

  
  max_vals = [0.6, 0.6, 0.7, np.pi, np.pi, np.pi]
  min_vals = [-0.6, -0.6, 0, -np.pi, -np.pi, -np.pi]

  for i in range(len(vars)):
    slider = FloatSlider(max=max_vals[i],
                             min=min_vals[i],
                             value=desired_pose[i],
                             step=0.01,
                             continuous_update=True,
                             description=vars[i],
                             layout=Layout(width='90%'))
    slider.observe(partial(slider_callback, index = i), names='value')
    display(slider)
    slider_widgets.append(slider)

  return slider_widgets

q_previous = None

In [11]:
def AddCollisionConstraints(ik, min_distance, arm_geometries, collidable_geometries):
  for i in range(len(arm_geometries)):
    for j in range(len(collidable_geometries)):
      pair = (arm_geometries[i], collidable_geometries[j])
      ik.AddDistanceConstraint(pair, min_distance, 1000) # we dont care about the upper bound so set it to a large number


def main():
    
  position_tolerance = 0.01

  panda_file = FindResourceOrThrow("drake/manipulation/models/franka_description/urdf/panda_arm.urdf")
  hand_file = FindResourceOrThrow("drake/manipulation/models/franka_description/urdf/hand.urdf")

  builder = DiagramBuilder()
  plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
  panda_model = Parser(plant, scene_graph).AddModelFromFile(panda_file, "panda")
  hand_model = Parser(plant, scene_graph).AddModelFromFile(hand_file)
  plant.WeldFrames(plant.world_frame(),plant.GetFrameByName("panda_link0"))
  plant.WeldFrames(plant.GetFrameByName("panda_link8"),plant.GetFrameByName("panda_hand"))

  c1 = AddCylinder(plant, "c1", 0.05, 0.5)
  plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("c1", c1), RigidTransform([0.15, 0.15, 0.25])) # add a model to test collisions

  plant.Finalize()

  meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)

  diagram = builder.Build()
  context = diagram.CreateDefaultContext()
  plant_context = plant.GetMyContextFromRoot(context)
  

  q_nominal = plant.GetPositions(plant_context, panda_model)
  q_previous = None
  end_effector_frame = plant.GetFrameByName('panda_hand')
  world_frame = plant.world_frame()
  console = Textarea(value="", description="", layout={'width':'60%','height':'100px'}, style={'description_width':'initial'})

  arm_geometries = []
  bodies = plant.GetBodyIndices(plant.GetModelInstanceByName("panda"))
  for i in bodies:
    b = plant.get_body(i)
    if (b.name() == "panda_link0"): # this is because the distance constraint is not supported between two static objects in the scene
        continue
    arm_geometries+=plant.GetCollisionGeometriesForBody(b)
        
  collidable_names = ["c1"]
  to_avoid_geometries = []
  for name in collidable_names:
    bodies = plant.GetBodyIndices(plant.GetModelInstanceByName(name))
    for i in bodies:
      to_avoid_geometries+=plant.GetCollisionGeometriesForBody(plant.get_body(i))

  def IK_callback(context, desired_pose):
    global q_previous
    '''
    ik =PandaInverseKinematics(plant, plant_context, panda_model)

    # position constraint
    p_tol = np.ones(3)*0.01
    ik.AddPositionConstraint(desired_pose[0:3]-p_tol, desired_pose[0:3]+ p_tol)
    #ik.AddPointToPointDistanceConstraint(end_effector_frame, np.array([0,0,0]), world_frame, desired_pose[0:3], 0, position_tolerance) # also can use AddPositionConstraint
    
    # add collision constraint
    # ik.AddMinimumDistanceConstraint(0.01, 0.1) # this should work but doesn't for this franka-panda model (it works for the IIWA)
    #AddCollisionConstraints(ik, 0.01,arm_geometries, to_avoid_geometries)

    # add orientation constraint
    rot = RigidTransform(RollPitchYaw(desired_pose[3:]), desired_pose[0:3]).rotation()
    #ik.AddOrientationConstraint(end_effector_frame, RotationMatrix(), world_frame, rot, 0)
    ik.AddOrientationConstraint(rot, 0.01)
    

    # set inital guess and solve
    prog = ik.get_prog()
    q = ik.get_q()
    '''
    p_tol = np.ones(3)*0.01
    trans = desired_pose[0:3]
    
    ik = InverseKinematics(plant, plant_context)
    ik.AddPositionConstraint(
            end_effector_frame,
            np.zeros(3),
            world_frame,
            trans - p_tol,
            trans + p_tol)
    rot = RigidTransform(RollPitchYaw(desired_pose[3:]), desired_pose[0:3]).rotation()
    ik.AddOrientationConstraint(
            end_effector_frame,
            RotationMatrix(),
            world_frame,
            rot,
            0)
    ik.AddMinimumDistanceConstraint(0.01, 0.1)
    q = ik.q()
    prog = ik.prog()
    q_nom = np.concatenate((q_nominal, np.zeros(2)))
    #prog.AddQuadraticErrorCost(np.identity(len(q)), q_nom, q)
    #prog.SetInitialGuess(q, q_nom)
        
    # prog.AddQuadraticErrorCost(np.identity(len(q)), q_nominal, q) # optional
    if (q_previous is None):
      prog.SetInitialGuess(q, q_nom)
    else:
      prog.SetInitialGuess(q, q_previous)

    result = Solve(prog)
    q_previous = result.GetSolution()
    console.value += str(result.is_success()) + "\n"
    plant.SetPositions(plant_context, panda_model, q_previous[:-2])
    
  meshcat.load()
  PoseWidget(meshcat, context, IK_callback)
  display(console)

In [12]:
main()

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6031...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7031/static/
Connected to meshcat-server.
